# 0. Importing the Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys, re, json, time
import itertools, collections
from importlib import reload
from IPython.display import display
import scipy.sparse
import nltk
from w266_common import utils, vocabulary, tf_embed_viz


/home/luisarmandovillarreal/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 1. Getting Tickers and Tweets

We have two different data folders ('data' and 'data2'). This is because we first got all the tweets that mentioned name of stock as per the convention $ (ticker). 

Afterwards we got tweets that include names of companies along with their ticker symbols in the tweets. For example, we gathered any tweet that mentioned Amazon, and any tweet that mention $ (AMZN). This substantially increased our tweet count.

Our tweets dataset consist of tweets pertaining to S&P500 Top 50 companies only.

In [51]:
# getting all the files in folder 'data'
files = []
for file in os.listdir('data'):
    if file.endswith('.csv'):
        files.append(os.path.join('data', file))

In [94]:
# creating dataframe from files in 'data' folder

data = []

for filename in files:
    with open(filename, 'r') as file:
        ticker = filename.split('_')[-1][:-4]
        line = file.readline()
        line = file.readline()
        while(line):
            ts =line.split(';"')[0][1:].split(';')[0]
            tweet = line.split(';"')[1].split('";')[0]
            tweet_id = line.split('/')[-1][:-1] 
            line=file.readline()
            data.append([tweet_id,ticker,ts,tweet])
        
#df = pd.DataFrame(data, columns=['tweet_id','ticker','timestamp','tweet'])
#df

In [53]:
# getting all the files in 'data2' folder
files1 = []
for file in os.listdir('data2'):
    if file.endswith('.csv'):
        files1.append(os.path.join('data2', file))

In [95]:
# creating dataframe for files in 'data2' folder


for filename in files1:
    with open(filename, 'r') as file:
        ticker = filename.split('_')[-1][:-4]
        line = file.readline()
        line = file.readline()
        while(line):
            ts =line.split(';"')[0][1:].split(';')[0]
            tweet = line.split(';"')[1].split('";')[0]
            tweet_id = line.split('/')[-1][:-1] 
            line=file.readline()
            data.append([tweet_id,ticker,ts,tweet])
        
df = pd.DataFrame(data, columns=['tweet_id','ticker','timestamp','tweet'])

As can be seen from shapes of dataframes below, our first dataframe has considerably less tweets than the second one.

In [23]:
print(df.shape)

(155011, 4)
(2989666, 4)


In [24]:
df.head()

,tweet_id,ticker,timestamp,tweet
0,1105569239845478401,DWDP,2019-03-12 20:40,4 nice trades today 3 swings ($csco $ dwdp $ m...
1,1105566065369722885,DWDP,2019-03-12 20:27,$ AAPL - scaled out 2/3 into strength; 1/3 sto...
2,1105548846279450624,DWDP,2019-03-12 19:19,SLD 100 $ DWDP @55.69
3,1105548512119283713,DWDP,2019-03-12 19:18,BOT $ DWDP Apr 55 call @2.34
4,1105521484640980995,DWDP,2019-03-12 17:30,"# estate asset watch - stocks with momentum, 5..."


In [25]:
companies_dict = {'Union':'UNP', 'Disney':'DIS', 'Nike':'NKE', 'Chevron':'CVX',
                  'Intel':'INTC', 'Bank':'BAC', 'Pepsi':'PEP', 'Amgen':'AMGN',
                  'AT&T':'T', 'Procter':'PG', 'Microsoft':'MSFT', 'Wells':'WFG',
                  'Walmart':'WMT', 'Citigroup':'C', 'Verizon':'VZ', 'Exxon-Mobil':'XOM',
                  'Apple':'AAPL', 'Mastercard':'MA', 'Merck':'MRK', 'Boeing':'BA', 
                  'Comcast':'CMCSA', 'Salesforce':'CRM', 'Home':'HD', 'Berkshire':'BRK',
                  'Cisco':'CSCO', 'ATT':'T', 'Dow':'DWDP', 'Coca-Cola':'KO', 'Visa':'V',
                  'Facebook':'FB', 'Johnson':'JNJ', 'Abbott':'ABBT', 'Broadcom':'AVGO',
                  '3M':'MMM', 'Pfizer':'PFE', 'Amazon':'AMZN', 'Honeywell':'HON', 'Adobe':'ADBE',
                  'Google':'GOOG', 'Netflix':'NFLX', 'Eli':'LLY', 'Phillips':'PM', 'United':'UNP',
                  'AbbVie':'ABBV', 'McDonald':'MCD', 'JP':'JPM', 'Paypal':'PYPL', 'Oracle':'ORCL', 
                  'Medtronic':'MDT'}

In [96]:
# Associate tickers with company names

for key, value in companies_dict.items():
    
    print(key,'       ', end='\r')
    df.loc[df['ticker'] == key, 'ticker'] = value
    #combined_df.at[index, 'ticker'] = companies_dict[row['ticker']]
    

In [27]:
# Get Nasdaq Stock Symbols
os.system("curl --ftp-ssl anonymous:jupi@jupi.com "
          "ftp://ftp.nasdaqtrader.com/SymbolDirectory/nasdaqlisted.txt "
          "> nasdaq.lst")

0

In [52]:
!head -43 nasdaq.lst

<!DOCTYPE html><html data-adblockkey="MFwwDQYJKoZIhvcNAQEBBQADSwAwSAJBANDrp2lz7AOmADaN8tA50LsWcjLFyQFcb/P2Txc58oYOeILb3vBw7J6f4pamkAQVSQuqYsKx3YzdUHCvbVZvFUsCAwEAAQ==_pxrolOu5yyzUS0zMNgm01mNbFq+3Njr33BJOzQ+JCg65FKwTyH/JswWy8wg/VocffrVzS1j7NXxyt+jJ2gFZsQ=="><head><meta http-equiv="Content-Type" content="text/html; charset=utf-8"><title></title><meta name="viewport" content="width=device-width, initial-scale=1"><meta name="description" content="See related links to what you are looking for."/></head><!--[if IE 6 ]><body class="ie6"><![endif]--><!--[if IE 7 ]><body class="ie7"><![endif]--><!--[if IE 8 ]><body class="ie8"><![endif]--><!--[if IE 9 ]><body class="ie9"><![endif]--><!--[if (gt IE 9)|!(IE)]> --><body><!--<![endif]--><script type="text/javascript">g_pb=(function(){var
DT=document,azy=location,DD=DT.createElement('script'),aAA=false,LU;DD.defer=true;DD.async=true;DD.src="//www.google.com/adsense/domains/caf.js";DD.onerror=function(){if(azy.search!=='?z'){azy.href='/?z';}};DD.onlo

In [53]:
!tail -10 nasdaq.lst

In [54]:
!tail -n +43 nasdaq.lst | cat | sed '$d' | sed 's/|/ /g' > nasdaq.lst2

In [55]:
!echo; head nasdaq.lst2; echo "..."; tail nasdaq.lst2












...












In [ ]:
!awk '{print $1}' nasdaq.lst2 > nasdaq.csv

In [28]:
tickers = pd.read_csv('nasdaq.csv', index_col=None, header=None)
tickers.columns = ['ticker']

In [29]:
# Get NYSE Symbols
nyse = pd.read_csv('companylist.csv')
cols = [1,2,3,4,5,6,7,8,9]
nyse.drop(nyse.columns[cols],axis=1,inplace=True)
nyse.columns = ['ticker']
tickers = tickers.append(nyse)
tickers = list(tickers['ticker'])

In [97]:
# Get Tweets associated with more than one stock...

# First, using the tweet id...
g = df.groupby('tweet_id').size().reset_index(name='count')
print('Tweets associated with more than one stock:',"{:,}".format(sum(g[g['count']>1]['count'])))

# Delete those tweet ids
tweets_ids_to_drop = list(g[g['count']>1]['tweet_id'])
df = df[~df.tweet_id.isin(tweets_ids_to_drop)]
df.shape

Tweets associated with more than one stock: 542,968


(2601709, 4)

In [16]:
# This one takes a while when run for the first time

import glove_helper; reload(glove_helper)

hands = glove_helper.Hands(ndim=100)

Loading vectors from data/glove/glove.6B.zip
Parsing file: data/glove/glove.6B.zip:glove.6B.100d.txt
Found 400,000 words.
Parsing vectors... Done! (W.shape = (400003, 100))


In [111]:
# Second, discard tweets that mention multiple stocks
# This one takes a while, hang in there
# Take advantage of the whole iteration and transform tweet for classifier
df_duplicate = df.copy()

TOKEN_RE = re.compile(r"\w.*?\b")
indices_to_drop = []

for index, row in df_duplicate.iterrows():
    
    print(index,end='\r')

    tweet_no_url = re.sub(r'http\S+', '', row['tweet'])
    tokens = tweet_no_url.split(' ')
    sentence = '<s> '
    flag = True
    prev_token = ''
    for token in tokens:
        try:
            a = hands.vocab.word_to_id[token.lower()]
            sentence = sentence + token.lower() + ' '

        except:
            pass

        if token in tickers and prev_token == '$':

            if token != row['ticker']:
                indices_to_drop.append(index)
                flag=False
                break
        
        prev_token = token
    
    sentence += '</s>'
    
    if flag:
        
        df_duplicate.at[index,'tweet'] = re.sub('[^A-Za-z0-9]+', '', sentence)

df_duplicate.drop(indices_to_drop, inplace=True)

,tweet_id,ticker,timestamp,tweet
24,110526365952428032,DWDP,2019-03-12 00:26,<s> traders sell shares of $ on strength $ </s>
25,1103805919568293888,T,2019-03-07 23:53,<s> @ ich $ @ ich @ </s>
26,1103801007375552514,T,2019-03-07 23:34,<s> $ house democrats seek details of trump ef...
56,1103735736153432064,T,2019-03-07 19:14,<s> $ t - two new revenue streams … … @ att @ ...
63,1103724496027115520,T,2019-03-07 18:30,<s> telus co. declares quarterly dividend of $...
103,1103672373990080517,T,2019-03-07 15:02,<s> a t & t $ t shareholder douglass winthrop ...
104,1103671152411635712,T,2019-03-07 14:58,<s> not an exciting stock but $ t is green - l...
178,1103556398334521344,T,2019-03-07 07:22,<s> $ content chief says competitors eating ou...
179,1103547615923392512,T,2019-03-07 06:47,<s> $ $ $ newmont rejects barrick takeover … </s>
180,1103547608113602560,T,2019-03-07 06:47,<s> $ $ high-speed rail in the u.s. remains il...


In [117]:
df_duplicate.to_csv('data/clean_tweets.csv', index=False)